# Import Schema

In [9]:
import pandas as pd
import json

print(pd.__version__)

# Create a DataFrame

schema_path = "schema/Attendance Schema - Workers.csv"

df = pd.read_csv(schema_path)

data = df.to_dict(orient='records')

print(data)

def convert_to_json_schema(df):
    schema = {
        "bsonType": "object",
        "required": [],
        "properties": {}
    }
    
    for _, row in df.iterrows():
        field_name = row['VARIABLE NAME']
        data_type = str(row['DATA TYPE'])  # Ensure data_type is a string
        description = row['DESCRIPTION']
        
        if pd.notna(row['VALIDATION RULES']):
            validation_rules = json.loads(row['VALIDATION RULES'])
        else:
            validation_rules = {}
        
        schema['properties'][field_name] = {
            "bsonType": data_type.lower(),
            "description": description
        }
        
        if field_name not in schema['required']:
            schema['required'].append(field_name)
        
        if validation_rules:
            schema['properties'][field_name].update(validation_rules)
    
    return schema

json_schema = convert_to_json_schema(df)

2.2.3
[{'VARIABLE NAME': '_id', 'DESCRIPTION': 'Worker ID', 'DATA TYPE': 'ObjectId', 'VALIDATION RULES': nan, 'COMMENT': 'Primary identifier'}, {'VARIABLE NAME': 'NAME', 'DESCRIPTION': 'Worker Name', 'DATA TYPE': 'String', 'VALIDATION RULES': nan, 'COMMENT': nan}, {'VARIABLE NAME': 'CITY', 'DESCRIPTION': 'Worker City', 'DATA TYPE': 'String', 'VALIDATION RULES': nan, 'COMMENT': nan}, {'VARIABLE NAME': 'ASSIGNED_CLIENT_ID', 'DESCRIPTION': 'Assigned Client', 'DATA TYPE': 'ObjectId', 'VALIDATION RULES': nan, 'COMMENT': 'Links to Clients _id'}, {'VARIABLE NAME': 'ATTENDANCE', 'DESCRIPTION': 'Attendance Data', 'DATA TYPE': 'Array[Attendance]', 'VALIDATION RULES': nan, 'COMMENT': nan}, {'VARIABLE NAME': 'STATUS', 'DESCRIPTION': 'Labourer Status', 'DATA TYPE': nan, 'VALIDATION RULES': '["Available", "Absent", "Assigned"]', 'COMMENT': nan}]


ValueError: dictionary update sequence element #0 has length 9; 2 is required

In [3]:
import pandas as pd
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get MongoDB URI from environment variables
MONGO_URI = os.getenv("MONGO_URI")

# Initialize MongoDB client and database
client = MongoClient(MONGO_URI)
db = client['Attendance_DB']

# Load the CSV data into a DataFrame
csv_file_path = "schema/sample_attendance_data.csv"
df = pd.read_csv(csv_file_path)

# Define a function to upload data to MongoDB
def upload_to_mongodb(df, collection_name):
    collection = db[collection_name]
    data = df.to_dict(orient='records')
    try:
        result = collection.insert_many(data)
        return result.inserted_ids
    except BulkWriteError as bwe:
        print(f"Bulk write error: {bwe.details}")
        return []

# Upload the data to the 'workers' collection
collection_name = "workers"
inserted_ids = upload_to_mongodb(df, collection_name)
print(f"Inserted document IDs: {inserted_ids}")

ModuleNotFoundError: No module named 'dotenv'